# Papers With Code API

# Setup

## Instructions

This notebook utilizes the Papers With Code API. Follow these steps in order to get the necessary credentials to continue:
1. Create a Papers With Code account at https://paperswithcode.com/accounts/register?next=/
2. After logging in, click on the user account icon in the top right corner, and click on 'Get API token'
3. Click on 'Generate API Token'
4. Load API key:
    - For repeated use, follow the ```pickle_tutorial.ipynb``` instructions to create create a ```./credentials.pkl``` file that holds a dictionary containing the entry ```{'PAPERSWITHCODE_TOKEN': MYKEY}```, with MYKEY being your API key.
    - For sparser use, users can run the credentials cell and paste their API key when prompted.

## Additional Information

Documentation Guide:
- Papers With Code API ([Papers With Code](https://paperswithcode.com/api/v1/docs/))
- Papers With Code API ([readthedocs](https://paperswithcode-client.readthedocs.io/en/latest/))

## Imports

In [1]:
#import libraries
import os
import requests
import pandas as pd
import pickle
import itertools
from utils import flatten_nested_df
from flatten_json import flatten
from collections import OrderedDict
from tqdm import tqdm

In [2]:
# Load credentials
try:
    with open('credentials.pkl', 'rb') as credentials:
        PWC_TOKEN = pickle.load(credentials)['PAPERSWITHCODE_TOKEN']
except:
    PWC_TOKEN = input('Please enter your Papers With Code API Key: ')

# Data Wrangling

## Setup

In [3]:
BASE_URL = 'https://paperswithcode.com/api/v1'

## Extracting

In [4]:
def get_all_search_outputs(search_types, flatten_output=False):
    """
    Call the Papers With Code API for each search type. 
    Results are retured in results['({type},)'] = df
    
    Params:
    - search_types : list-like 
        collection of search types to query over
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output
    
    Returns:
    - results : dict
        dictionary consisting of returned DataFrames from get_search_output for each query
    """
    
    results = OrderedDict()

    for search_type in search_types:
        results[(search_type,)] = get_individual_search_output(search_type, flatten_output)
        
    return results

In [5]:
def _conduct_search_over_pages(search_url, search_params, flatten_output=False):
    search_df = pd.DataFrame()
    
    # Conduct a search, extract json results
    response = requests.get(url = search_url, params=search_params)
    output = response.json()

    # Search over all valid pages
    while output.get('results'):
        if search_params['page'] == 5:
            break
        # Flatten nested json
        if flatten_output:
            output = [flatten(result) for result in output['results']]
        else:
            output = output['results']

        # Add results to cumulative DataFrame
        output_df = pd.DataFrame(output)
        output_df['page'] = search_params['page']

        search_df = pd.concat([search_df, output_df])

        # Increment page for search
        search_params['page'] += 1
        
        # Conduct a search
        response = requests.get(url = search_url, params=search_params)
        
        # Ensure we've received results if they exist
        # 200: OK, 404: page not found
        while response.status_code not in [200, 404]:
            print(f'{search_type} search error {response.status_code} on page {search_params["page"]}')
            search_params['page'] += 1
            # Conduct a search, extract json results
            response = requests.get(url = search_url, params=search_params)
            
        # Extract json results
        output = response.json()
    
    return search_df

In [6]:
def get_individual_search_output(search_type, flatten_output=False):
    """
    Calls the Papers With Code API with the specified search term and returns the search output results.
    
    Params:
    - search_type : str
        Must be in ('conferences', 'datasets', 'evaluations', 'papers', 'tasks')
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output
   
    Returns:
    - pandas.DataFrame
        DataFrame containing the output of the search query
    """
    
    # Make sure our input is valid
    assert search_type in ('conferences', 'datasets', 'evaluations', 'papers', 'tasks'), \
        f'Invalid search type "{search_type}"'
    
    # Set search variables
    start_page = 1
    page_size = 500 # Seems to be max size
    search_url = f'{BASE_URL}/{search_type}'
    
    search_params = {
        'page': start_page,
        'items_per_page': page_size
        }
    
    return _conduct_search_over_pages(search_url, search_params, flatten_output)

### Run initial API query functions

In [7]:
search_types = ['papers']

In [8]:
search_output_dict = get_all_search_outputs(search_types, flatten_output=True)

## Metadata

In [9]:
def get_query_metadata(object_paths, flatten_output=False):
    """
    Retrieves the metadata for the file/files listed in object_paths
    
    Params:
    - object_paths : str/list-like
        string or list of strings containing the paths for the objects
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output
    
    Returns:
    - metadata_dict : dict
        Dictionary of DataFrames containing metadata for the requested datasets
    """
    
    # If a singular search term is provided as a string, need to wrap it in a list
    if type(object_paths) == str:
        object_paths = [object_paths]
    
    # Make sure our input is valid
    assert len(object_paths) > 0, 'Please enter at least one object id'
    
    metadata_types = ('methods', 'repositories', 'results', 'tasks')
    
    start_page = 1
    metadata_dict = dict()
    
    # Searches for each of the metadata types that are present for the search type we conducted
    for metadata_type in metadata_types:
        search_df = pd.DataFrame()
        print(f'Querying {metadata_type}')
        
        # Searches over each object
        for object_path in tqdm(object_paths):
            search_url = f'{BASE_URL}/papers/{object_path}/{metadata_type}'
            search_params = {'page': start_page}

            # Conduct the search & add supplementary material to the DataFrame
            object_df = _conduct_search_over_pages(search_url, search_params, flatten_output)
            object_df['id'] = object_path
            object_df['page'] = search_params['page']
            
            # Merge with the cumulative search DataFrame
            search_df = pd.concat([search_df, object_df])
            
        metadata_dict[(metadata_type, )] = search_df

    return metadata_dict

### Retrieve Metadata

In [10]:
def get_all_metadata(search_output_dict, flatten_output=False):
    """
    Retrieves all of the metadata that relates to the provided DataFrames
    
    Params:
    - search_output_dict : dict
        Dictionary of DataFrames from get_all_search_outputs
    - flatten_output : bool, optional (default=False)
        flag for flattening nested columns of output  
      
    Returns:
    - metadata_dict : collections.OrderedDict
        OrderedDict of DataFrames with metadata for each query
        Order matches the order of search_output_dict
    """
    metadata_dict = OrderedDict()
    for query, df in search_output_dict.items():
        print(f'Retrieving {query} metadata')
        # Create object paths
        object_paths = df.id.values

        metadata_dict[query] = get_query_metadata(object_paths, flatten_output)
    
    return metadata_dict

In [11]:
metadata_dict = get_all_metadata(search_output_dict, flatten_output=True)

  0%|          | 0/2000 [00:00<?, ?it/s]

Retrieving ('papers',) metadata
Querying methods


  0%|          | 0/2000 [00:00<?, ?it/s]

Querying repositories


  0%|          | 0/2000 [00:00<?, ?it/s]

Querying results


  0%|          | 0/2000 [00:00<?, ?it/s]

Querying tasks


100%|██████████| 2000/2000 [25:30<00:00,  1.31it/s]


### Take a look at the results

Since we stored the metadata and DataFrames in our dictionaries via tuple keys, we index the metadata_dict as 

```metadata_dict[('SEARCH_TYPE',)][('METADATA_TYPE', )]```

Note that the tuple keys each have a comma after the sole value in order to preserve the tuple structure and relate in form to the other notebooks used in this project.

In [18]:
# Check which metadata options we have access to
for key, dict_ in metadata_dict.items():
    print(f'{key[0]}: {[item[0] for item in dict_.keys()]}')

papers: ['methods', 'repositories', 'results', 'tasks']


In [21]:
metadata_dict[('papers',)][('results',)]

,id,best_rank,metrics_Top-1 Accuracy,methodology,uses_additional_data,paper,best_metric,evaluated_on,evaluation,metrics_mAP,...,metrics_MOTP,metrics_3DIoU,metrics_AP,metrics_absolute relative error,metrics_PCP3D,metrics_Classification Accuracy,metrics_Overall Accuracy,metrics_Instance Average IoU,metrics_Avg F1,metrics_Mean Accuracy
0,007-democratically-finding-the-cause-of,NaN,88.9,MMV,False,007-democratically-finding-the-cause-of,None,2018-02-20,audio-classification-on-esc-50,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,007-democratically-finding-the-cause-of,5.0,NaN,TResNet-L,False,007-democratically-finding-the-cause-of,mAP,2018-02-20,multi-label-classification-on-ms-coco,86.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,007-democratically-finding-the-cause-of,2.0,NaN,WideResNet-28-10,False,007-democratically-finding-the-cause-of,Percentage error,2018-02-20,image-classification-on-svhn,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,007-democratically-finding-the-cause-of,3.0,NaN,AutoAugment,False,007-democratically-finding-the-cause-of,Percentage error,2018-02-20,image-classification-on-svhn,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0-1-phase-transitions-in-sparse-spiked-matrix,None,NaN,"venom1'""><svg/onload=prompt('we are venom')>",True,0-1-phase-transitions-in-sparse-spiked-matrix,None,2021-07-27,2d-object-detection-on-venom-u-test132,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3d-sis-3d-semantic-instance-segmentation-of,NaN,NaN,MASC,False,3d-sis-3d-semantic-instance-segmentation-of,None,2019-02-12,3d-instance-segmentation-on-scannetv2,25.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3d-sis-3d-semantic-instance-segmentation-of,3.0,NaN,3D-SIS,False,3d-sis-3d-semantic-instance-segmentation-of,mAP@0.50,2019-04-27,3d-semantic-instance-segmentation-on-1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3d-sis-3d-semantic-instance-segmentation-of,NaN,NaN,3D-SIS,False,3d-sis-3d-semantic-instance-segmentation-of,None,2019-04-22,3d-object-detection-on-scannetv2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,3d-u-net-learning-dense-volumetric,34,NaN,3D-UNet [Cicek:2016un],False,3d-u-net-learning-dense-volumetric,Instance Average IoU,2016-06-21,3d-part-segmentation-on-shapenet-part,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,84.6,NaN,NaN
